## barebones efficientnet HPA classification training

## credits to:
 https://www.kaggle.com/samusram/hpa-rgb-model-rgby-cell-level-classification/

for the generator code

https://www.kaggle.com/glopezzz/segmentation-and-classification-with-efficientnet

for efficientnet training code

future things:
2 models - r+g (only microtubules) and r+b (only nucleus) - would have to re-code the labels 
 
if we decide to use the current image size (512x512) we can download the resized images locally and have less preprocessing during training. This might also have the added benefit of having augmented images in the same folder for better access during training. 
 

currently, the model seems to always predict the vector of all zeros - is this a product of the label matrix being sparse??? so predicting all 0 is technically pretty "accurate".

this can very well be that I really didn't train the model all that much.
EDIT 0419: yeah. it wasn't actually trained. accuracy still quite bad, but at least it's giving 1s and 0s.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#### install. segmentations and tf_explain only comes into play during "submission" phase so we leave it out for now. Unless later on we want to segment before training.

In [ ]:
!pip install "../input/keras-application/Keras_Applications-1.0.8-py3-none-any.whl"
!pip install "../input/efficientnet111/efficientnet-1.1.1-py3-none-any.whl"
# !pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
# !pip install "../input/hpapytorchzoozip/pytorch_zoo-master"
# !pip install "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"
# !pip install "../input/tfexplainforoffline/tf_explain-0.2.1-py3-none-any.whl"

#### import. Some are probably redundant for this run 

In [ ]:
import os, glob
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# tf.compat.v1.disable_eager_execution()
from copy import deepcopy
import random
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import Callback
# note from OP: please note, that locally I've trained a keras.efficientnet model, but using tensorflow.keras.applications.EfficientNetB0 should lead to the same results
from efficientnet.keras import EfficientNetB0
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from keras.utils import Sequence
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
import matplotlib.pyplot as plt
from IPython.display import Image, display
from numpy.random import seed
seed(10)
from tensorflow.python.framework import ops
import gc
from tqdm.auto import tqdm
import base64
import numpy as np
# from pycocotools import _mask as coco_mask
import typing as t
import warnings
warnings.filterwarnings('ignore')
tf.random.set_seed(10)
%matplotlib inline

### setup

In [ ]:
TEST_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/test/'
TRAIN_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/train/'
IMG_HEIGHT = IMG_WIDTH = 512
BATCH_SIZE = 32

# internet must be enabled
DOWNLOAD_PRETRAINED_WEIGHTS = True

train_df = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')

### splarghing with the class names

In [ ]:
# from https://www.kaggle.com/c/hpa-single-cell-image-classification/data

specified_class_names = """0. Nucleoplasm
1. Nuclear membrane
2. Nucleoli
3. Nucleoli fibrillar center
4. Nuclear speckles
5. Nuclear bodies
6. Endoplasmic reticulum
7. Golgi apparatus
8. Intermediate filaments
9. Actin filaments 
10. Microtubules
11. Mitotic spindle
12. Centrosome
13. Plasma membrane
14. Mitochondria
15. Aggresome
16. Cytosol
17. Vesicles and punctate cytosolic patterns
18. Negative"""

class_names = [class_name.split('. ')[1] for class_name in specified_class_names.split('\n')]

In [ ]:
train_df['Label'] = train_df['Label'].map(lambda x: set(map(int, x.split('|'))))
for class_i, class_name in enumerate(class_names):
    train_df[class_name] = train_df['Label'].map(lambda x: 1 if class_i in x else 0)

In [ ]:
train_df.head(3)

In [ ]:
# dictionary for image ID --> ohe vectors
id_2_ohe_vector = {img: vec for img, vec in zip(train_df['ID'], train_df.iloc[:, 2:-1].values)}

### separate the images with unique label combinations for training

In [ ]:
label_combinations = train_df['Label'].map(lambda x: str(sorted(list(x))))
print("There {} images with unique label combinations ".format(sum(label_combinations.value_counts()==1)))
label_combinations_counts = label_combinations.value_counts()
unique_label_combination = label_combinations_counts.index[label_combinations_counts==1]

In [ ]:
train_ids_unique_label_comb = train_df['ID'][train_df['Label'].map(lambda x:str(sorted(list(x))) in unique_label_combination)]
non_unique_label_comb_bool_idx = train_df['Label'].map(lambda x:str(sorted(list(x))) not in unique_label_combination)

In [ ]:
train_ids, val_ids = train_test_split(train_df['ID'][non_unique_label_comb_bool_idx].values,
                        test_size = 0.2,
                        stratify = label_combinations[non_unique_label_comb_bool_idx],
                        random_state = 42)
train_ids = np.concatenate((train_ids, train_ids_unique_label_comb))

In [ ]:
print(len(train_ids),len(val_ids))

## data is too large to fit into memory, so we need a generator

heavily borrowed from kaggle user samusram's code. Most major change is replacing the num_channels argument with use_colors, a string of colors, eg "rgb", "rgby" etc. 
Would need to make sure that in future segmentation and cell-level classification, the channels are loaded in the same order. 

Currently, the model in this notebook is 3-channel, so it defaults to rgb. Any other channel count will require remaking the model

also need to test if shuffle works since it was taken without editing

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, id_list, id_2_ohe_vector=id_2_ohe_vector, folder_imgs=TRAIN_IMGS_FOLDER, 
                 batch_size=BATCH_SIZE, shuffle=True,
                 resized_height=IMG_HEIGHT, resized_width=IMG_WIDTH, use_colors = "rgb"):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.id_list = deepcopy(id_list)
        self.folder_imgs = folder_imgs
        self.len = len(self.id_list) // self.batch_size
        self.resized_height = resized_height
        self.resized_width = resized_width
        self.id_2_ohe_vector = id_2_ohe_vector
        self.is_test = not 'train' in folder_imgs
        self.use_colors = use_colors
        self.num_channels = len(use_colors)
        if not self.is_test:       
            self.num_classes = len(next(iter(id_2_ohe_vector.values())))
        if not shuffle and not self.is_test:
            self.labels = [id_2_ohe_vector[img] for img in self.id_list[:self.len*self.batch_size]]

    def __len__(self):
        return self.len
    
    def on_epoch_start(self):
        if self.shuffle:
            random.shuffle(self.id_list)
            
    # open_image adapted from https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb
    def open_image(self, image_id): #a function that reads an image with the specified colors
        
        color_dict = {"r":"red","g":"green","b":"blue","y":"yellow"}
        colors = [color_dict[i] for i in self.use_colors]
        img = [cv2.imread(os.path.join(self.folder_imgs, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE)
               for color in colors]
        img = np.stack(img, axis=-1)
        if img.shape[0] == self.resized_height and img.shape[1] == self.resized_width:
            return img
        img_resized = cv2.resize(img, (self.resized_height, self.resized_width))
        # no need to expand dims here 
        
        return img_resized
    # btw id_list is the entire batch of IDs! so just pass train_ids or val_ids here. 
    # idx seems to be the batch number, created internally. 
    def __getitem__(self, idx):
        current_batch = self.id_list[idx * self.batch_size: (idx + 1) * self.batch_size]
        X = np.empty((self.batch_size, self.resized_height, self.resized_width, self.num_channels))

        if not self.is_test:
            y = np.empty((self.batch_size, self.num_classes))

        for i, image_id in enumerate(current_batch):
            img = self.open_image(image_id)
            X[i, :, :, :] = img.astype(np.float32)/255.0
            if not self.is_test:
                y[i, :] = self.id_2_ohe_vector[image_id]
        if not self.is_test:
            return X, y
        return X
    # an extra function?? maybe only used in testing. 
    def get_labels(self):
        if self.shuffle:
            images_current = self.id_list[:self.len*self.batch_size]
            labels = [self.id_2_ohe_vector[img] for img in images_current]
        else:
            labels = self.labels
        return np.array(labels)

#### functions to grab image data from the id -  more for looking at the data purposes. Not used for training anymore.

In [ ]:
def grab_training_img(img_idx,use_colors):
    color_dict = {"r":"red","g":"green","b":"blue","y":"yellow"}
    image_id = train_ids[img_idx]
    
    colors = [color_dict[i] for i in use_colors]
    
    img = [cv2.resize(cv2.imread(os.path.join(TRAIN_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE),
                      (IMG_HEIGHT, IMG_WIDTH))
            for color in colors]
    img = np.stack(img, axis=-1)
    return img, id_2_ohe_vector[image_id]

def grab_val_img(img_idx,use_colors):
    color_dict = {"r":"red","g":"green","b":"blue","y":"yellow"}
    image_id = val_ids[img_idx]
    colors = [color_dict[i] for i in use_colors]
    img = [cv2.resize(cv2.imread(os.path.join(TRAIN_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE),
                      (IMG_HEIGHT, IMG_WIDTH))
            for color in colors]
    img = np.stack(img, axis=-1)
    return img, id_2_ohe_vector[image_id]


In [ ]:
plt.imshow(grab_val_img(6,"rgb")[0])

# make the model
## barebones EfficientNet. Plenty of params to tune here including the base model
### sigmoid was recommended by the internet

output is 1 less than the number of class names, since if a prediction is 0 for everything, the label would be the "Negative" label.

# EDIT 04/19: trained a model under rgb-effnet-512h5. can be reloaded and trained further.

In [ ]:
weights_init = 'imagenet' if DOWNLOAD_PRETRAINED_WEIGHTS else None

# make the parts
imagenet_model = EfficientNetB0(weights=weights_init, include_top=False, pooling='avg',
                               input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rgb_model_output = Dense(len(class_names) - 1, activation='sigmoid')(imagenet_model.output)

# make the model
model_rgb = Model(inputs=imagenet_model.input, outputs=rgb_model_output)

In [ ]:
model_rgb.summary()

some old code. scrapped bc data couldn't fit into memory.

In [ ]:
# training_data = []
# for i in range(len(train_ids)):
#     img,label = grab_training_img(i,"rgb")
#     img = np.expand_dims(img, 0)
#     label = label.reshape(1,18)
#     training_data.append([img,label])
    

# train_ds = tf.data.Dataset.from_tensor_slices(([training_data[i][0] for i in range(len(training_data))], [training_data[i][1] for i in range(len(training_data))]))
# len(train_ds)

# val_data = []
# for i in range(val_ids):
#     img,label = grab_val_img(i,"rgb")
#     img = np.expand_dims(img, 0)
#     label = label.reshape(1,18)
#     val_data.append([img,label])

# val_ds = tf.data.Dataset.from_tensor_slices(([val_data[i][0] for i in range(len(val_data))], [val_data[i][1] for i in range(len(val_data))]))
# len(val_ds)

### compile the model with basic suggestions:
from a brief google search, binary_crossentropy is recommended for multi-label since increasing one label's score doesn't penalize others. Can be played around with 

metrics also requires some care, as some of them only check pred == ground truth, hence considering all partially correct classifications to be wrong


In [ ]:
# also grabbed from sources without question. 
tf.keras.backend.clear_session()

earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, verbose=0, mode='min',
    restore_best_weights=True
)

model_rgb.compile('adam', 'binary_crossentropy', metrics=[tf.keras.metrics.AUC(multi_label=True)])
# model_rgb.compile('adam', 'binary_crossentropy', metrics=["accuracy"])

# artifact of copypasta - maybe a way to use a cluster?
#run = wandb.init(entity='usernamehere', project='hpa', job_type='train')

generate data, then train. 
Takes a while. At least the training initializes and runs, so that's that. 

In [ ]:
BATCH_SIZE = 32

In [ ]:
# run generator
my_training_batch_generator = DataGenerator(train_ids,shuffle=True)
my_validation_batch_generator = DataGenerator(val_ids,shuffle=True)

In [ ]:
##### this can be added to the fitter I think:
#   callbacks=[earlystopper]

# drastically truncated for demonstration. At steps_per_epoch = 500, takes 2-3 hours per epoch. 
# with GPU, 12 mins per epoch with 250 steps.

hist = model_rgb.fit_generator(generator=my_training_batch_generator,
                               steps_per_epoch = int(len(train_ids)//BATCH_SIZE),
                               epochs=10,
                               validation_data = my_validation_batch_generator,
                               validation_steps = int(len(val_ids)//BATCH_SIZE),
                               verbose=1
                )

In [ ]:
model_rgb.save("rgb_effnet_full_train_0420.h5")

model can then be used for further purposes.

In [ ]:
look_idx = 123

img,label = grab_val_img(look_idx,"rgb")
preds= model_rgb.predict(np.expand_dims(img,0))
print(label)
print(np.round(preds[0],1))